---
title: Core
subtitle: Core data structures and algorithms
---


In [ ]:
# | default_exp core
# | export
from pydantic import ConfigDict, field_validator
from pydantic.dataclasses import dataclass
from datetime import datetime

## Common Utilities

In [ ]:
# | exporti
def list2dict(variables: list):
    return {v.name: v for v in variables}


def normalize(v):
    """Utility function to convert a list to a dictionary"""
    return list2dict(v) if isinstance(v, list) else v


def normalize_t(type, v):
    """Utility function to convert a list of `type` to a dictionary"""
    return (
        list2dict(v)
        if isinstance(v, list) and all(isinstance(x, type) for x in v)
        else v
    )


def add_v(self, name, v):
    return getattr(self, name).update({v.name: v})

## Variable

In [ ]:
# | export
@dataclass(config=ConfigDict(extra="allow"))
class Variable:
    name: str = None
    description: str = None
    unit: str = None
    timerange: list[datetime] = None


@dataclass
class Variables:
    timerange: list[datetime] = None
    variables: dict[str, Variable] = None

    def add_variable(self, v):
        add_v(self, "variables", v)

## Dataset

In [ ]:
# | export
@dataclass
class Dataset(Variables):
    name: str = None

## Instrument and Mission

In [ ]:
# | export
@dataclass
class Instrument:
    name: str
    datasets: dict[str, Dataset] = field_validator("datasets", mode="before")(normalize)

    names_list: list[str] = None

    def add_dataset(self, dataset: Dataset):
        self.datasets[dataset.name] = dataset


@dataclass
class Mission:
    name: str
    """Name of the mission"""
    instruments: dict[str, Instrument] = field_validator("instruments", mode="before")(
        normalize
    )
    datasets: dict[str, Dataset] = None

    names_list: list[str] = None

    def add_instrument(self, v):
        add_v(self, "instruments", v)